In [1]:
import torch
import datetime
import torchvision
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import requests
import tarfile
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset
import zipfile
from PIL import Image, ImageEnhance
import torchvision.transforms.functional as TF

import random

torch.manual_seed(4)

In [2]:
# define transformations that will be applied to all the datasets
transform = transforms.Compose([
    transforms.Resize((120, 120)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # normalize to [-1, 1]
    transforms.Lambda(lambda x: x / 2 + 0.5)  # normalize to [0, 1]
])

In [3]:
# function to filter out images with a certain class
def filter_classes(dataset, class_lbl):
    indices = []
    classes = dataset.classes
    for i, (_, label) in enumerate(dataset):
      if label in [classes.index(class_lbl)]:
        indices.append(i)
    return indices

# Collect Data

### Upload the Oxford-IIIT Pet Dataset

In [4]:
'''
raw_train_dataset = torchvision.datasets.OxfordIIITPet(root='./data/oxford-pets', download=True)
raw_test_dataset = torchvision.datasets.OxfordIIITPet(root='./data/oxford-pets', split='test', download=True)

# Extract breed names from file paths
breeds = sorted(os.listdir(os.path.join('./data/oxford-pets/oxford-iiit-pet', 'images')))
new_breeds = []
for breed in breeds:
    # Remove numbers and '.jpg' from each breed name
    breed_name = ''.join(filter(lambda x: not x.isdigit(), breed))
    breed_name = breed_name.replace('.jpg', '')
    breed_name = breed_name.replace('_', ' ')
    breed_name = breed_name.replace('.mat', '')

    # Append "yorkshire terrier 175.jpg" to the new list
    new_breeds.append(breed_name)

new_breeds = list(set(new_breeds))  # Remove duplicates
new_breeds.sort()
# Print the modified list

for breed in new_breeds:
    print(breed)

cat_breeds = new_breeds[:12]
dog_breeds = new_breeds[12:]
'''

'\nraw_train_dataset = torchvision.datasets.OxfordIIITPet(root=\'./data/oxford-pets\', download=True)\nraw_test_dataset = torchvision.datasets.OxfordIIITPet(root=\'./data/oxford-pets\', split=\'test\', download=True)\n\n# Extract breed names from file paths\nbreeds = sorted(os.listdir(os.path.join(\'./data/oxford-pets/oxford-iiit-pet\', \'images\')))\nnew_breeds = []\nfor breed in breeds:\n    # Remove numbers and \'.jpg\' from each breed name\n    breed_name = \'\'.join(filter(lambda x: not x.isdigit(), breed))\n    breed_name = breed_name.replace(\'.jpg\', \'\')\n    breed_name = breed_name.replace(\'_\', \' \')\n    breed_name = breed_name.replace(\'.mat\', \'\')\n\n    # Append "yorkshire terrier 175.jpg" to the new list\n    new_breeds.append(breed_name)\n\nnew_breeds = list(set(new_breeds))  # Remove duplicates\nnew_breeds.sort()\n# Print the modified list\n\nfor breed in new_breeds:\n    print(breed)\n\ncat_breeds = new_breeds[:12]\ndog_breeds = new_breeds[12:]\n'

### Extracting "cats" and "dogs" classes from CIFAR10:



In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# filter out images with cats and dogs only
cat_indices = filter_classes(trainset, 'cat')
dog_indices = filter_classes(trainset, "dog")

# Create a subset containing only images with cats and dogs
cat_subset = torch.utils.data.Subset(trainset, cat_indices)
dog_subset = torch.utils.data.Subset(trainset, dog_indices)
print("The CIFAR10 dataset downloaded successfully.")

#os.remove('./data/cifar-10-python.tar.gz')

100%|██████████| 170498071/170498071 [00:04<00:00, 40215678.29it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
The CIFAR10 dataset downloaded successfully.


### Extracting "dogs" classes from Stanford dataset:


In [6]:
_URL = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'
tar_file_path = './data/stanford-dogs.tar'

response = requests.get(_URL)
with open(tar_file_path, 'wb') as f:
    f.write(response.content)

extract_dir = './data/stanford-dogs'
with tarfile.open(tar_file_path, 'r') as tar:
    tar.extractall(extract_dir)

os.remove(tar_file_path)

print("The Stanford Dogs dataset downloaded and extracted successfully.")

The Stanford Dogs dataset downloaded and extracted successfully.


In [7]:
stanford_dogs_dataset = ImageFolder(root='./data/stanford-dogs/Images', transform=transform)
#classes = stanford_dogs_dataset.classes
stanford_dogs_dataset.classes = 'dog'

for i in range(len(stanford_dogs_dataset)):
    _, label = stanford_dogs_dataset.samples[i]
    stanford_dogs_dataset.samples[i] = stanford_dogs_dataset.samples[i][0], 'dog'

### Concatenating dogs datasets

In [8]:
dogs = ConcatDataset([stanford_dogs_dataset, dog_subset])
random_indices = torch.randperm(len(dogs))
cats = torch.utils.data.Subset(dogs, random_indices)
print(len(dogs))

25580


In [9]:
# check dimensionality

dataloader = DataLoader(dogs, batch_size=1, shuffle=False)
dataset_size = len(dogs)
print("Size of the concatenated dataset:", dataset_size)

for image, _ in dataloader:
    sample_image_dimensions = image.shape[1:]  # Ignore batch dimension
    break

print("Dimensions of a sample image:", sample_image_dimensions)

Size of the concatenated dataset: 25580
Dimensions of a sample image: torch.Size([3, 120, 120])


### Extracting "Cats" class from kaggle set

In [10]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_file_path = './data/cats_and_dogs_filtered.zip'

response = requests.get(_URL)
with open(zip_file_path, 'wb') as f:
    f.write(response.content)
extract_dir = './data'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

os.remove(zip_file_path)

print("Cats Kaggle dataset downloaded and extracted successfully.")

Cats Kaggle dataset downloaded and extracted successfully.


In [11]:
cats_dogs_train = ImageFolder(root='./data/cats_and_dogs_filtered/train', transform=transform)

cat_indices = filter_classes(cats_dogs_train, 'cats')

cat_subset_2 = torch.utils.data.Subset(cats_dogs_train, cat_indices)
print(len(cat_subset_2))

1000


### Extracting "Cats" class from Microsoft Cats&Dogs dataset

In [12]:
_URL = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
zip_file_path = './data/kagglecatsanddogs.zip'

response = requests.get(_URL)
with open(zip_file_path, 'wb') as f:
    f.write(response.content)
extract_dir = './data'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

os.remove(zip_file_path)

print("Microsoft cats and dogs dataset downloaded and extracted successfully.")

Microsoft cats and dogs dataset downloaded and extracted successfully.


In [13]:
cats_dogs_train_3 = ImageFolder(root='./data/PetImages', transform=transform)
classes = cats_dogs_train_3.classes

cat_indices = []
for idx in range(len(cats_dogs_train_3)):
    try:
        img, label = cats_dogs_train_3[idx]
        if label == classes.index('Cat'):
            cat_indices.append(idx)
    except Exception as e:
        print(f"Skipped corrupted file at index {idx}: {e}")


# create a subset containing only cat images
cat_subset_3 = torch.utils.data.Subset(cats_dogs_train_3, cat_indices)
print(len(cat_subset_3))


Skipped corrupted file at index 8790: cannot identify image file <_io.BufferedReader name='./data/PetImages/Cat/666.jpg'>
Skipped corrupted file at index 14395: cannot identify image file <_io.BufferedReader name='./data/PetImages/Dog/11702.jpg'>


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


12499


### Concatenating cats datasets

In [14]:
cats = ConcatDataset([cat_subset, cat_subset_2, cat_subset_3])
random_indices = torch.randperm(len(cats))
cats = torch.utils.data.Subset(cats, random_indices)
print(len(cats))

18499


In [15]:
# check dimensionality

dataloader = DataLoader(cats, batch_size=1, shuffle=False)
dataset_size = len(cats)
print("Size of the concatenated dataset:", dataset_size)
for image, _ in dataloader:
    sample_image_dimensions = image.shape[1:]  # Ignore batch dimension
    break

print("Dimensions of a sample image:", sample_image_dimensions)

Size of the concatenated dataset: 18499
Dimensions of a sample image: torch.Size([3, 120, 120])


### Concatenating cats and dogs datasets and ensure equal sizes

In [16]:
dogs_reduced = torch.utils.data.Subset(dogs, torch.randperm(len(cats)))
pets = ConcatDataset([cats, dogs_reduced])
pets = torch.utils.data.Subset(pets,torch.randperm(len(pets)))

print(len(pets))

36998


### Saving data for pretraining in a separate folder

In [17]:
output_dir = "./pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    filename = f"image_{i}.jpg"
    image_path = os.path.join(output_dir, filename)
    image_pil.save(image_path)

print("Images saved successfully.")

Images saved successfully.


#Implement Transformation Functions

### Rotation Function

In [18]:
# function to randomly rotate an image and return the rotation degree
def rotate_image(image):
    degrees = [0, 90, 180, 270]
    deg = random.choice(degrees)
    rotated_image = image.rotate(deg, expand=True)
    return rotated_image, deg


In [19]:
# run if you want to apply rotation

'''
output_dir = "./rotated_pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    rotated_image, degree = rotate_image(image_pil)

    # save the image with a unique filename including the rotation degree
    filename = f"image_{i}_{degree}deg.jpg"
    image_path = os.path.join(output_dir, filename)
    rotated_image.save(image_path)

print("Images saved successfully with labels indicating rotation degree.")
'''

'\noutput_dir = "./rotated_pets_images"\nos.makedirs(output_dir, exist_ok=True)\n\nfor i, (image, label) in enumerate(pets):\n    image_pil = transforms.ToPILImage()(image)\n    rotated_image, degree = rotate_image(image_pil)\n\n    # save the image with a unique filename including the rotation degree\n    filename = f"image_{i}_{degree}deg.jpg"\n    image_path = os.path.join(output_dir, filename)\n    rotated_image.save(image_path)\n\nprint("Images saved successfully with labels indicating rotation degree.")\n'

### Color Distortion function

In [20]:
# function to apply color distortion to an image and return the distorted image
def distort_color(image):
    color_jitter = transforms.ColorJitter(
        brightness=0.5,
        contrast=0.5,
        saturation=0.5,
        hue=0.1
    )

    distorted_image = color_jitter(image)
    return distorted_image


In [21]:
# run if you want to apply color jittering
'''
output_dir = "./distorted_pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    distorted_image = distort_color(image_pil)

    # save the distorted image
    filename = f"image_{i}_distorted.jpg"
    image_path = os.path.join(output_dir, filename)
    distorted_image.save(image_path)

print("Images saved successfully with color distortion.")
'''

'\noutput_dir = "./distorted_pets_images"\nos.makedirs(output_dir, exist_ok=True)\n\nfor i, (image, label) in enumerate(pets):\n    image_pil = transforms.ToPILImage()(image)\n    distorted_image = distort_color(image_pil)\n\n    # save the distorted image\n    filename = f"image_{i}_distorted.jpg"\n    image_path = os.path.join(output_dir, filename)\n    distorted_image.save(image_path)\n\nprint("Images saved successfully with color distortion.")\n'

### Greyscale Function

In [22]:
# function to apply random greyscale transformation to an image
def greyscale(image):
    grayscale_transform = transforms.RandomGrayscale(p=0.2)
    greyscale_image = grayscale_transform(image)

    return greyscale_image

In [23]:
# run if you want to apply greyscaling
'''
output_dir = "./greyscaled_pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    distorted_image = greyscale(image_pil)

    # save the distorted image
    filename = f"image_{i}_greyscaled.jpg"
    image_path = os.path.join(output_dir, filename)
    distorted_image.save(image_path)

print("Images saved successfully with greyscale.")
'''

'\noutput_dir = "./greyscaled_pets_images"\nos.makedirs(output_dir, exist_ok=True)\n\nfor i, (image, label) in enumerate(pets):\n    image_pil = transforms.ToPILImage()(image)\n    distorted_image = greyscale(image_pil)\n\n    # save the distorted image\n    filename = f"image_{i}_greyscaled.jpg"\n    image_path = os.path.join(output_dir, filename)\n    distorted_image.save(image_path)\n\nprint("Images saved successfully with greyscale.")\n'

### Masking Function

In [24]:
def mask_image(image, mask_percentage):

    rows, cols = image.shape[1:]
    rows_to_mask = int(rows * mask_percentage / 100)
    cols_to_mask = int(cols * mask_percentage / 100)

    top_left_row = random.randint(0, rows - rows_to_mask)
    top_left_col = random.randint(0, cols - cols_to_mask)

    image[:, top_left_row:top_left_row + rows_to_mask, top_left_col:top_left_col + cols_to_mask] = 0.5

    return image

In [25]:
# run if you want to apply masking
'''
output_dir = "./masked_pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    masked_image = mask_image(image, 75)
    masked_image = transforms.ToPILImage()(masked_image)
    # save the masked image
    filename = f"image_{i}_masked.jpg"
    image_path = os.path.join(output_dir, filename)
    masked_image.save(image_path)

print("Images saved successfully with greyscale.")
'''

Images saved successfully with greyscale.


### Apply all transformations

In [26]:
'''
output_dir = "./transformed_pets_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(pets):
    image_pil = transforms.ToPILImage()(image)
    im = greyscale(image_pil)
    im = distort_color(im)
    im, rotation = rotate_image(im)

    # save the distorted image
    filename = f"image_{i}_transformed.jpg"
    image_path = os.path.join(output_dir, filename)
    im.save(image_path)

print("Transformed images saved successfully.")
'''

'\noutput_dir = "./transformed_pets_images"\nos.makedirs(output_dir, exist_ok=True)\n\nfor i, (image, label) in enumerate(pets):\n    image_pil = transforms.ToPILImage()(image)\n    im = greyscale(image_pil)\n    im = distort_color(im)\n    im, rotation = rotate_image(im)\n\n    # save the distorted image\n    filename = f"image_{i}_transformed.jpg"\n    image_path = os.path.join(output_dir, filename)\n    im.save(image_path)\n\nprint("Transformed images saved successfully.")\n'